In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Essentially a simplified, modified version of tinkertytonk project - so credit to him

In [0]:
!pip install kaggle --upgrade -q

In [0]:
import os
import gc
import numpy as np
# import jovian
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy, error_rate
from fastai.callbacks import *

from sklearn.metrics import f1_score

from PIL import Image
from tqdm.notebook import tqdm 
from pathlib import Path

Setup Kaggle API Key

In [0]:
os.environ['KAGGLE_USERNAME']="skr1125"
os.environ['KAGGLE_KEY']=""

In [0]:
!pwd

In [0]:
!ls ../input

In [0]:
# setting up paths for use in the notebook
PATH = '../input/human-protein-atlas-image-classification/'
TRAIN = '../input/human-protein-atlas-image-classification/train/'
TEST =  '../input/human-protein-atlas-image-classification/test/'
LABELS = '../input/human-protein-atlas-image-classification/train.csv'
path_working = Path('/kaggle/working')

In [0]:
channels4 = ['_yellow', '_red', '_green_', '_blue']
channels3 = ['_red', '_green', '_blue']

In [0]:
index_class_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

The following two cells are really NOT needed for the process of visualizing or conducting the training. Just an illustration to show the file and process it into a one hot vector of structures present in the training images.

In [0]:
# Read the training data so that the cell id and the structures they contain (as strings) can be seen in the Target field
train_df = pd.read_csv(LABELS) # see LABELS definition above
train_df.head()

In [0]:
# next convert the whole thing into a one hot vector coding as well 
for i in range(28):
    train_df[f'{index_class_dict[i]}'] = train_df['Target'].map(lambda x:1 if str(i) in x.strip().split() else 0)
train_df.head()

The cells below need to be repeated for the 4 channel case. I am trying this first for a simple 3 channel model

In [0]:
# Suppose we only use the rgb values provided initially and IGNORE the y values, so only 3 channels 
# We will try 4 channels after that - remember to initialize y value with avg of other 3 NOT zeros
# import Fastai vision to get their Image class
from fastai.vision.image import *

# taken from : https://github.com/wdhorton/protein-atlas-fastai/blob/master/utils.py
# discussion : https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/71039
# adapted from https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb
def open_3_channel(fname):
    fname = str(fname)
    # strip extension before adding color
    if fname.endswith('.png'):
        fname = fname[:-4]
    # SKR: colors below changed to only be 3 colors red, green, blue 
    # SKR: IGNORING YELLOW for now
    #colors = ['red','green','blue','yellow']
    colors = ['red', 'green', 'blue']
    flags = cv2.IMREAD_GRAYSCALE
    
    img = [cv2.imread(fname+'_'+color+'.png', flags).astype(np.float32)/255
           for color in colors]
    
    # convert from a [512,512,4] tensor to a [4,512,512] tensor
    # convert from a [512, 512, 3] tensor to a [3, 512, 512] tensor
    x = np.stack(img, axis=-1)    
    
    # create a Fastai image from the tensor
    return Image(pil2tensor(x, np.float32).float())

Create the DataBunch. Here another approach could be to size the images to be 224 x 224 above and train the model first 
THEN train the model on 512 x 512 images and use that model to predict. 

In [0]:
bs=16
size=512

In [0]:
# read submission file to get the names of test images
test_df = pd.read_csv(PATH + 'sample_submission.csv')
test_df.head()

In [0]:
np.random.seed(230)

In [0]:
PATH

In [0]:
test = ImageList.from_df(test_df, PATH, folder='test', suffix='.png')

In [0]:
src = (ImageList.from_df(train_df, PATH, folder='train', suffix='.png')
                .split_by_rand_pct(0.2)
                .label_from_df(cols='Target', label_delim=' ')
                .add_test(test))

In [0]:
src.train.x.create_func = open_3_channel
src.train.x.open = open_3_channel

src.valid.x.create_func = open_3_channel
src.valid.x.open = open_3_channel

In [0]:
src.test.x.create_func = open_3_channel
src.test.x.open = open_3_channel

In [0]:
# 4 channel protein stats - going to only 3 channel stats
# protein_stats = ([0.08069, 0.05258, 0.05487, 0.08282], [0.13704, 0.10145, 0.15313, 0.13814])
protein_stats = ([0.08069, 0.05258, 0.05487], [0.13704, 0.10145, 0.15313])

Adding transforms here using
get_transforms(do_flip:bool=True, flip_vert:bool=False, max_rotate:float=10.0, max_zoom:float=1.1, max_lighting:float=0.2, max_warp:float=0.2, p_affine:float=0.75, p_lighting:float=0.75, xtra_tfms:Optional[Collection[Transform]]=None) → Collection[Transform]

In [0]:
trn_tfms,_ = get_transforms(do_flip=True, flip_vert=True, max_rotate=30.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.1, p_affine=0.5, p_lighting=0.5)

In [0]:
# create databunch after using bs and applying transforms and normalizing using protein stats 
# data = src.databunch(bs=bs, dl_tfms=tfms).normalize(protein_stats)
data = (src.transform((trn_tfms, _), size=512)
        .databunch(bs=16).normalize(protein_stats))

In [0]:
type(data)

In [0]:
data.show_batch(rows=3, figsize=(12,9))

In [0]:
data.c

In [0]:
arch = models.resnet34

This should be modified to be different metrics as per competition spec

In [0]:
acc_02 = partial(accuracy_thresh, thresh=0.2)

Check what the fbeta with thresh 0.2 gives for f_score and its relationship to definition of F1 score 

In [0]:
# Continue to use this as well
f_score = partial(fbeta, thresh=0.2)

Trying to define a F1 class for accumulating stats on each batch and calculating F1 score for entire val dataset. 
References:
https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb
https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
https://www.kaggle.com/guglielmocamporese/macro-f1-score-keras
https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354

So generate a metric F1 and a loss which correlates with the F1 metric and use them in the learner. 


In [0]:
# Am able to get the focal loss piece working but 
# Am not able to get the F1 macro metric working nor the F1 macro loss


class F1:
  
  __name__ = 'F1 macro'

  def __init__(self, n=28):
    self.n = n
    self.TP = torch.zeros(self.n)
    self.FP = torch.zeros(self.n)
    self.FN = torch.zeros(self.n)

  def __call__(self, preds, targs, thresh=0.2):
    preds = (preds > thresh).int() # Check whether this matters if it is NOT float
    targs = targs.int()
    self.TP += (preds*targs).float().sum(dim=0)
    self.FP += (preds > targs).float().sum(dim=0)
    self.FN += (preds < targs).float().sum(dim=0)
    score = (2.0 * self.TP/(2.0 * self.TP + self.FP + self.FN + 1e-6)).mean()
    return score

  def reset(self):
    score = (2.0 * self.TP/(2.0 * self.TP + self.FP + self.FN + 1e-6))
    print('F1 macro:', score.mean(), flush=True)
    self.TP = np.zeros(self.n)
    self.FP = np.zeros(self.n)
    self.FN = np.zeros(self.n)

class F1_callback(Callback):
  def __init__(self, n=28):
    self.f1 = F1(n)

  def on_epoch_end(self, metrics):
    self.f1.reset()

class F1_Loss(nn.Module):

  def __init__(self, n=28):
    self.n = n
    self.TP = torch.zeros(self.n)
    self.FP = torch.zeros(self.n)
    self.FN = torch.zeros(self.n)

  def forward(self, preds, targs, thresh=0.2):
    preds = (preds > thresh).int() # Check whether this matters if it is NOT float
    targs = targs.int()
    self.TP += (preds*targs).float().sum(dim=0)
    self.FP += (preds > targs).float().sum(dim=0)
    self.FN += (preds < targs).float().sum(dim=0)
    score = (2.0 * self.TP/(2.0 * self.TP + self.FP + self.FN + 1e-6))
    # need this to keep score from becoming too small or too big
    score = score.clamp(min=1e-6, max= 1 - 1e-6)
    return 1 - score.mean()

# Do we need this - commenting it out at first
# f1_loss = F1_Loss().cuda()

def focal_loss(input, target, reduction='mean', beta=0.5, gamma=2., eps=1e-7, **kwargs):
    n = input.size(0)
    iflat = torch.sigmoid(input).view(n, -1).clamp(eps, 1-eps)
    tflat = target.view(n, -1)
    focal = -(beta*tflat*(1-iflat).pow(gamma)*iflat.log() + 
             (1-beta)*(1-tflat)*iflat.pow(gamma)*(1-iflat).log()).mean(-1)
    if torch.isnan(focal.mean()) or torch.isinf(focal.mean()):
        pdb.set_trace()
    if reduction == 'mean':
        return focal.mean()
    elif reduction == 'sum':
        return focal.sum()
    else:
        return focal
    
class FocalLoss(nn.Module):
    def __init__(self, beta=0.5, gamma=2., reduction='mean'):
        super().__init__()
        self.beta = beta
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input, target, **kwargs):
        focal = focal_loss(input, target, beta=self.beta, gamma=self.gamma, reduction=self.reduction, **kwargs)
        return focal


In [0]:
# learn = cnn_learner(data, arch, metrics=[acc_02, f_score])
# modifying above to only give it data and arch
learn = cnn_learner(data, arch)

In [0]:
# Do we need F1_loss.cuda() here 
# Trying it with cuda first did not work so switched
# That did not work so trying with Focal_Loss first
# learn.loss_func = F1_Loss()
learn.loss_func = FocalLoss()

In [0]:
f1_callback = F1_callback()
learn.metrics = [acc_02, f1_callback.f1]

In [0]:
learn.summary()

Need to set learn.model_dir attribute in Learner to a full libpath path that is writable and so 

In [0]:
learn.model_dir = path_working 

In [0]:
path_working

In [0]:
learn.model_dir = path_working

In [0]:
learn.model_dir

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
lr = 3e-2 

In [0]:
learn.fit_one_cycle(5, slice(lr))

In [0]:
# not executing as running out of gpu for this week 
# learn.fit_one_cycle(5, slice(lr))

In [0]:
learn.save('stage1-rn34-3chaugs-focloss')

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
# reducing cycles to fit GPU Quota
learn.fit_one_cycle(2, slice(1e-5, lr/5))

In [0]:
learn.save('stage2-rn34-3chaugs-focloss')

Try below with test time augmentation - so learner.TTA(DatasetType.Test, beta=0.4, scale=1.35)

In [0]:
#preds, _ = learn.get_preds(DatasetType.Test)
preds, _ = learn.TTA(ds_type=DatasetType.Test, beta=0.4, scale=1.35)

In [0]:
type(preds)

In [0]:
preds.shape

In [0]:
type(learn.data.classes)

In [0]:
len(learn.data.classes)

In [0]:
thresh = 0.2
labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds]

In [0]:
len(labelled_preds)

In [0]:
labelled_preds[:5]

In [0]:
learn.data.test_ds.items[0]

In [0]:
Path(learn.data.test_ds.items[0]).stem

In [0]:
# converting image path strings to the file name only with no extension
fnames = [Path(f).stem for f in learn.data.test_ds.items]

In [0]:
sample_list = list(test_df.Id)
# sample_list[:5]
pred_dict = dict((key, value) for (key, value) in zip(fnames, labelled_preds))
pred_list_cor = [pred_dict[id] for id in sample_list]
df = pd.DataFrame({'ID':sample_list, 'Predicted':pred_list_cor})
df.to_csv('protein_classification.csv', header=True, index=False)

In [0]:
df.head()

In [0]:
# Submit
!kaggle competitions submit -c human-protein-atlas-image-classification -f protein_classification.csv -m "FocalLoss 3chAugs"

# View results
!kaggle competitions submissions -c human-protein-atlas-image-classification > results.txt

In [0]:
!ls results.txt

In [0]:
!cat results.txt